In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import torch
import torch.nn as nn
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

c:\Users\zhang\anaconda3\envs\tensorflow\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data Generation

In [2]:
NUM_USER = 10000

In [3]:
NUM_Product = 10

In [4]:
treatment_percentage = 0.5

In [5]:
discount = 0.2

In [6]:
user_continuous_feature_multiplier = 1

In [7]:
prod_continuous_feature_multiplier = 1

In [8]:
# Set constants
USER_Cont_FEATURES = 2*user_continuous_feature_multiplier
USER_Dicr_FEATURES = 3

Product_Cont_FEATURES = 3*prod_continuous_feature_multiplier
Product_Dicr_FEATURES = 2
OUTSIDE_OPTION_UTILITY = 0
utilities = torch.zeros(NUM_USER, NUM_Product)

In [9]:
def generate_features(N, C, D):
    continuous_features = np.zeros((N, C))
    for i in range(C):
        continuous_features[:, i] = np.random.uniform(0,1,size=N)
    binary_features = np.random.randint(0, 2, (N, D))
    return np.hstack((continuous_features, binary_features))

In [10]:
import torch.nn as nn
import torch.nn.init as init

class UtilityDNN(nn.Module):
    def __init__(self, user_features, product_features):
        super(UtilityDNN, self).__init__()
        self.fc1 = nn.Linear(user_features + product_features, 1)
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.uniform_(self.fc1.weight, a=-0.0, b=0.5)
    def forward(self, x):
        x = self.fc1(x)
        return x

class PriceSensitivityDNN(nn.Module):
    def __init__(self, user_features):
        super(PriceSensitivityDNN, self).__init__()
        self.fc1 = nn.Linear(user_features,1)
        self.fc2 = nn.Linear(1, 1)

        nn.init.constant_(self.fc1.weight, 0)
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.constant_(self.fc2.weight, 0)
    def forward(self, x):

        x = self.fc1(x)
        x = self.fc2(x)
        return torch.abs(x)


In [11]:
def utility_model(x_user, X_product, price, user_randomization, prod_randomization,pair_utility_model,price_sensitivity_model,gumbel_noise):
    num_users = x_user.shape[0]
    num_products = X_product.shape[0]
    

    price_sensitivities = price_sensitivity_model(x_user)

    for i in range(num_users):

        for j in range(num_products):
            # Determine if the user and product are in the treatment group
            is_user_treated = (user_randomization[i] == 1)
            is_product_treated = (prod_randomization[j] == 1)

            # Adjust price based on the experiment conditions
            adjusted_price = price[j] * discount if is_user_treated or is_product_treated else price[j]
            combined_features = torch.cat((x_user[i], X_product[j]), 0)
            utility_from_dnn = pair_utility_model(combined_features)
            price_effect = price_sensitivities[i] * adjusted_price

            utilities[i, j] = utility_from_dnn - price_effect + gumbel_noise[i,j]

    return utility_from_dnn,price_effect

In [12]:
def make_decision(utilities):
    num_users = utilities.shape[0]
    decisions = torch.zeros(num_users, dtype=torch.long)  
    for i in range(num_users):
        max_utility, chosen_product = torch.max(utilities[i], dim=0)

        # Compare the maximum utility with the outside option (utility = 0)
        if max_utility <= 0:
            decisions[i] = -1 
        else:
            decisions[i] = chosen_product 

    return decisions

In [13]:
def calculate_revenue(decisions, prices):
    total_revenue = 0.0

    # Iterate over each decision and add the corresponding product price to total revenue
    for i, decision in enumerate(decisions):
        if decision != -1:  # Check if the decision is not the outside option
            total_revenue += prices[decision].item()  # Add the price of the chosen product

    return total_revenue

In [14]:
X_user = generate_features(NUM_USER,USER_Cont_FEATURES, USER_Dicr_FEATURES)
X_product = generate_features(NUM_Product, Product_Cont_FEATURES, Product_Dicr_FEATURES)
price = np.random.uniform(0.5 ,1, NUM_Product)

X_user = torch.from_numpy(X_user).float()
X_product = torch.from_numpy(X_product).float()
price = torch.from_numpy(price).float()
gumbel_dist = torch.distributions.Gumbel(0, 1)
gumbel_noise = gumbel_dist.sample((NUM_USER, NUM_Product))

In [15]:
price

tensor([0.9095, 0.9339, 0.9683, 0.5530, 0.5794, 0.5476, 0.6965, 0.8665, 0.8784,
        0.7379])

In [16]:
pair_utility_model = UtilityDNN(X_user.shape[1], X_product.shape[1])
price_sensitivity_model = PriceSensitivityDNN(X_user.shape[1])

In [17]:
layer_weights = pair_utility_model.fc1.weight.data
layer_biases = pair_utility_model.fc1.bias.data

print("Weights:", layer_weights)
print("Biases:", layer_biases)

Weights: tensor([[0.1622, 0.2144, 0.4507, 0.2663, 0.1491, 0.0533, 0.0337, 0.1800, 0.1470,
         0.1712]])
Biases: tensor([0.])


In [18]:

new_biases = torch.from_numpy(np.array(1.0))

In [19]:
# Assign new weights and biases
with torch.no_grad():  # Avoid tracking this operation in the computation graph
    price_sensitivity_model.fc2.bias.copy_(new_biases)
    

In [20]:
layer_weights = price_sensitivity_model.fc2.weight.data
layer_biases = price_sensitivity_model.fc2.bias.data

print("Weights:", layer_weights)
print("Biases:", layer_biases)

Weights: tensor([[0.]])
Biases: tensor([1.])


In [21]:
import torch

def utility_model_batched(x_user, X_product, price, user_randomization, prod_randomization, pair_utility_model, price_sensitivity_model, gumbel_noise, batch_size=10):
    num_users = x_user.shape[0]
    num_products = X_product.shape[0]
    decisions = torch.zeros(num_users, dtype=torch.long)  # Initialize decision array

    # Convert numpy arrays to tensors if necessary
    if isinstance(user_randomization, np.ndarray):
        user_randomization = torch.from_numpy(user_randomization).to(torch.bool)
    if isinstance(prod_randomization, np.ndarray):
        prod_randomization = torch.from_numpy(prod_randomization).to(torch.bool)
    if isinstance(price, np.ndarray):
        price = torch.from_numpy(price)

    # Compute price sensitivities outside the batch loop
    price_sensitivities = price_sensitivity_model(x_user)

    # Iterate over users in batches
    for i in range(0, num_users, batch_size):
        batch_end = min(i + batch_size, num_users)  # Define the end of the batch
        batch_indices = slice(i, batch_end)  # Slice for batch indexing

        # Repeat the product features and price for each user in the batch
        batch_user_features = x_user[batch_indices].unsqueeze(1).expand(-1, num_products, -1)
        batch_prod_features = X_product.unsqueeze(0).expand(batch_end - i, -1, -1)
        batch_price = price.unsqueeze(0).expand(batch_end - i, -1)

        # Handle treatment adjustments in batch
        batch_user_treatment = user_randomization[batch_indices].unsqueeze(1).expand(-1, num_products) == 1
        batch_prod_treatment = prod_randomization.unsqueeze(0).expand(batch_end - i, -1) == 1
        batch_adjusted_price = torch.where(batch_user_treatment | batch_prod_treatment, batch_price * discount, batch_price)

        # Combine user and product features
        combined_features = torch.cat((batch_user_features, batch_prod_features), dim=2)

        # Compute utilities using the neural network in a batch
        utility_from_dnn = pair_utility_model(combined_features.view(-1, combined_features.shape[-1])).view(batch_end - i, num_products)
        price_effect = price_sensitivities[batch_indices] * batch_adjusted_price
        batch_utilities = utility_from_dnn - price_effect + gumbel_noise[batch_indices]
        max_utilities, chosen_products = torch.max(batch_utilities, dim=1)

        # Compare the maximum utility with the outside option (utility = 0)
        outside_option = -1 * torch.ones_like(chosen_products, dtype=torch.long)  # Match dimension and dtype
        decisions[batch_indices] = torch.where(max_utilities > 0, chosen_products, outside_option)

    return decisions


# GTE

## All treated scenario: all products are discounted

In [22]:
user_randomization = np.random.choice([0,1], NUM_USER, p=[1, 0])
prod_randomization = np.random.choice([0,1], NUM_Product, p=[0, 1])

In [23]:
decisions_all_treat=utility_model_batched(X_user, X_product,price, user_randomization, prod_randomization, 
                                          pair_utility_model, price_sensitivity_model, gumbel_noise, batch_size=10)

print("Decisions per user (product index or -1 for outside option):\n", decisions_all_treat)

Decisions per user (product index or -1 for outside option):
 tensor([5, 5, 4,  ..., 1, 2, 7])


In [24]:
all_num_unique = torch.unique(decisions_all_treat).numel()
print(all_num_unique)

10


In [25]:
for i in range(-1,10):
    print(torch.sum(decisions_all_treat==i))

tensor(0)
tensor(935)
tensor(961)
tensor(1116)
tensor(1082)
tensor(1182)
tensor(888)
tensor(849)
tensor(981)
tensor(771)
tensor(1235)


In [26]:
total_revenue_all_treated = calculate_revenue(decisions_all_treat, price*discount)
print(f"Total revenue from sales when all products are discounted: ${total_revenue_all_treated:.2f}")

Total revenue from sales when all products are discounted: $1525.56


## All control scenario: all products remain the original price

In [27]:
user_randomization = np.random.choice([0,1], NUM_USER, p=[1, 0])
prod_randomization = np.random.choice([0,1], NUM_Product, p=[1, 0])

decisions_all_control =utility_model_batched(X_user, X_product,price, user_randomization, prod_randomization, 
                                          pair_utility_model, price_sensitivity_model, gumbel_noise, batch_size=10)

print("Decisions per user (product index or -1 for outside option):\n", decisions_all_control)
total_revenue_all_control = calculate_revenue(decisions_all_control, price)
print(f"Total Revenue from Sales: ${total_revenue_all_control:.2f}")

Decisions per user (product index or -1 for outside option):
 tensor([5, 5, 4,  ..., 1, 6, 7])
Total Revenue from Sales: $7430.04


In [28]:
revenue_difference = total_revenue_all_treated - total_revenue_all_control
print(f"Revenue Difference (ALLTreated - ALLControl): ${revenue_difference:.2f}")
# print(f"Revenue Relative Difference (ALLTreated - ALLControl)/AllControl: {100*revenue_difference/total_revenue_all_control:.2f}%")

Revenue Difference (ALLTreated - ALLControl): $-5904.48


In [29]:
true = revenue_difference

## product randomization

In [30]:
def calculate_product_revenue(decisions, prices, prod_randomization):
    revenue_treated = 0.0
    revenue_control = 0.0

    # Iterate over each user's decision
    for user_index, decision in enumerate(decisions):
        if decision != -1:  # If the user chose a product
            product_price = prices[decision].item()  # Get the price of the chosen product

            # Check if the product was in the treatment or control group
            if prod_randomization[decision] == 1:
                revenue_treated += product_price
            else:
                revenue_control += product_price

    return revenue_treated, revenue_control

In [31]:
utilities = torch.zeros(NUM_USER, NUM_Product)
user_randomization = np.random.choice([0,1], NUM_USER, p=[1, 0])
prod_randomization = np.random.choice([0,1], NUM_Product, p=[1-treatment_percentage, treatment_percentage])
# prod_randomization = np.random.choice([0,1], NUM_Product, p=[1, ])
decisions_product_randomization =utility_model_batched(X_user, X_product,price, user_randomization, prod_randomization, 
                                          pair_utility_model, price_sensitivity_model, gumbel_noise, batch_size=10)

In [32]:
revenue_treated, revenue_control = calculate_product_revenue(decisions_product_randomization, price-price*(1-discount)*prod_randomization, prod_randomization)
naive = revenue_treated/treatment_percentage - revenue_control/(1-treatment_percentage)
print(f"Revenue from Treated Products: ${revenue_treated:.2f}")
print(f"Revenue from Control Products: ${revenue_control:.2f}")
print(f"Revenue Difference (Treated - Control) by naive DIM: ${naive:.2f}")
# print(f"Revenue Relative Difference (ALLTreated - ALLControl)/AllControl: {100*revenue_difference/revenue_control:.2f}%")

Revenue from Treated Products: $1081.14
Revenue from Control Products: $2095.83
Revenue Difference (Treated - Control) by naive DIM: $-2029.39


## Prepare training and testing data given experiment data

In [33]:
X_user_1, X_user_2, decision_1, decision_2 = train_test_split(
X_user, decisions_product_randomization, test_size=1/2, random_state=3407)


In [34]:
train_set = {
    'features': X_user_1,
    'labels': decision_1
}

test_set = {
    'features': X_user_2,
    'labels': decision_2
}

# Flag to switch between training and test set
use_train_set = False  # Set to False for the test set

# Function to get the current active dataset
def get_active_dataset(use_train):
    return train_set if use_train else test_set
def get_test_dataset(use_train):
    return test_set if use_train else train_set
# Retrieve the current dataset based on the flag
current_dataset = get_active_dataset(use_train_set)
X_user_train = current_dataset['features']
decision_train = current_dataset['labels']
X_user_test = get_test_dataset(use_train_set)['features']
decision_test =  get_test_dataset(use_train_set)['labels']

# use simple MNL structural model

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim

class LinearMNLModel(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(LinearMNLModel, self).__init__()
        # Initialize parameters for user and product features
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0)) 

    def forward(self, x_user, X_product, price, user_randomization, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand user and product features to create a [N, M, F] shaped tensor for each
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1).detach()
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1).detach()


        # Calculate linear utility from features
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Adjust prices based on randomization
        adjusted_price = torch.where(
             prod_randomization.unsqueeze(0),
            price * discount,  
            price
        )

        # Calculate utility from price, properly expanding its dimension
        utility_price = adjusted_price * self.beta_price  # [M]
        utility_price = utility_price.expand(N, M)  # [N, M]

        # Total utility including features and price
        total_utility = utility_user + utility_product + utility_price

        # Incorporate the outside option with utility 0
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)

        return utilities_with_outside





In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_user_train, X_product, price = X_user_train.to(device), X_product.to(device), price.to(device)
if isinstance(user_randomization, np.ndarray):
    user_randomization = torch.from_numpy(user_randomization).to(X_user_train.device).bool()
if isinstance(prod_randomization, np.ndarray):
    prod_randomization = torch.from_numpy(prod_randomization).to(X_user_train.device).bool()

decision_train = decision_train.long().to(device)

In [37]:
model = LinearMNLModel(user_feature_dim=USER_Cont_FEATURES+USER_Dicr_FEATURES,
                       product_feature_dim=Product_Cont_FEATURES+Product_Dicr_FEATURES).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [38]:


num_epochs = 10000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    utilities = model(X_user_train, X_product, price, user_randomization, prod_randomization)
    choice_probabilities = nn.functional.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), decision_train+1])
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')



Epoch 0, Loss: 2.7480978965759277
Epoch 1000, Loss: 2.2523162364959717
Epoch 2000, Loss: 2.248289108276367
Epoch 3000, Loss: 2.2470858097076416
Epoch 4000, Loss: 2.246516704559326
Epoch 5000, Loss: 2.246188163757324
Epoch 6000, Loss: 2.245976209640503
Epoch 7000, Loss: 2.245830774307251
Epoch 8000, Loss: 2.2457263469696045
Epoch 9000, Loss: 2.2456493377685547


In [39]:
choice_probabilities

tensor([[-10.0155,  -2.2636,  -2.9214,  ...,  -2.8097,  -2.2555,  -2.4124],
        [-13.4486,  -2.2636,  -2.9213,  ...,  -2.8096,  -2.2554,  -2.4124],
        [-11.2641,  -2.2636,  -2.9214,  ...,  -2.8097,  -2.2554,  -2.4124],
        ...,
        [-22.2856,  -2.2636,  -2.9213,  ...,  -2.8096,  -2.2554,  -2.4124],
        [-13.3430,  -2.2636,  -2.9213,  ...,  -2.8096,  -2.2554,  -2.4124],
        [-18.0846,  -2.2636,  -2.9213,  ...,  -2.8096,  -2.2554,  -2.4124]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

In [40]:
beta_price_est = model.beta_price.cpu().detach().numpy()

In [41]:
print(beta_price_est)

-1.0846511


In [42]:
model.beta_user

Parameter containing:
tensor([10.8519,  7.3817,  4.4586,  3.6662,  3.9259], device='cuda:0',
       requires_grad=True)

In [43]:
import torch
import torch.nn.functional as F

In [44]:
all_product_control = np.random.choice([0,1], NUM_Product, p=[1, 0])
all_product_treated = np.random.choice([0,1], NUM_Product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()

X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

utilities = model(X_user_test, X_product, price, user_randomization, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities

# Calculate expected revenue
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

utilities = model(X_user_test, X_product, price, user_randomization, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities

# Calculate expected revenue
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)*discount
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")


Expected Revenue: $3712.32
Expected Revenue: $764.05


In [45]:
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear*2
print(f"Revenue Difference (Treated - Control) by Linear MNL: ${linear:.2f}")
print(f"Absolute Percentage Estimation Error of Linear MNL:  {100*np.abs(linear-revenue_difference)/revenue_difference:.2f}%")


Revenue Difference (Treated - Control) by Linear MNL: $-5896.54
Absolute Percentage Estimation Error of Linear MNL:  -0.13%


# Use NMNL

In [46]:
X_user_train1, X_user_val, decision_train1,decision_val = train_test_split(X_user_train,decision_train,test_size=0.1,random_state=34)

In [47]:
def prepare_data(user_features, product_features, prices):
    num_products = product_features.shape[0]
    all_x_other_products = []
    for i in range(num_products):
        indices = [j for j in range(num_products) if j != i]
        other_products = product_features[indices].reshape(-1)
        all_x_other_products.append(other_products)

    # Convert lists to tensor
    all_x_other_products = torch.stack(all_x_other_products, dim=0)
  

    return user_features, product_features, prices, all_x_other_products


In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LinearNestedMNL(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(LinearNestedMNL, self).__init__()
        
  
        total_feature_dim = user_feature_dim + product_feature_dim + 1 
        self.utility_linear = nn.Linear(total_feature_dim, 1)
        self.raw_lambda = nn.Parameter(torch.tensor(2.0)) 

    def forward(self, x_user, x_product, prices):
 
        N = x_user.shape[0]
        M = x_product.shape[0]

        prices_expanded = prices.view(1, M, 1).expand(N, -1, -1)
        
      
        combined_features = torch.cat((
            x_user.unsqueeze(1).expand(-1, M, -1),       # (N, M, U_dim)
            x_product.unsqueeze(0).expand(N, -1, -1),    # (N, M, P_dim)
            prices_expanded                              # (N, M, 1)
        ), dim=2)


        utilities = self.utility_linear(combined_features).squeeze(-1)

       
        lam = torch.sigmoid(self.raw_lambda) 


        scaled_utilities = utilities / lam
        inclusive_value_log = torch.logsumexp(scaled_utilities, dim=1, keepdim=True) 
        
        v_buy = lam * inclusive_value_log
        v_outside = torch.zeros(N, 1, device=x_user.device)
        
        # Log Softmax over the two Nests
        nest_logits = torch.cat([v_buy, v_outside], dim=1)
        nest_log_probs = F.log_softmax(nest_logits, dim=1) 
        
        log_prob_buy_nest = nest_log_probs[:, 0].unsqueeze(1)
        log_prob_outside = nest_log_probs[:, 1].unsqueeze(1)

        log_prob_item_given_buy = scaled_utilities - inclusive_value_log
        final_log_probs_products = log_prob_item_given_buy + log_prob_buy_nest
        
        # Return: [Log P(Outside), Log P(Prod 1), ..., Log P(Prod M)]
        return torch.cat([log_prob_outside, final_log_probs_products], dim=1)

In [49]:
price = price.to(device)
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount) * prod_randomization))
user_features, product_features, prices, all_x_other_products = prepared_data
user_features.shape, product_features.shape, prices.shape, all_x_other_products.shape

(torch.Size([4500, 5]),
 torch.Size([10, 5]),
 torch.Size([10]),
 torch.Size([10, 45]))

In [50]:

model = LinearNestedMNL(
    user_feature_dim=X_user_train1.shape[1], 
    product_feature_dim=X_product.shape[1]
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


best_val_loss = float('inf')
patience = 15
patience_counter = 0


for epoch in range(1000):
    model.train()  
    optimizer.zero_grad()

   
    log_probs = model(user_features, product_features, prices)
    loss = -torch.mean(log_probs[torch.arange(log_probs.shape[0]), decision_train1 + 1])

    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_log_probs = model(X_user_val, product_features, prices)
        val_loss = -torch.mean(val_log_probs[torch.arange(val_log_probs.shape[0]), decision_val + 1])
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Training Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}")

    # Early Stopping Check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0 
    else:
        patience_counter += 1 

    if patience_counter >= patience:
        print(f"Early stopping triggered at Epoch {epoch}")
        break

print(f"Final Lambda (Nesting Parameter): {torch.sigmoid(model.raw_lambda).item():.4f}")


def calculate_expected_revenue(model, user_features, product_features, prices):
    model.eval()
    with torch.no_grad():
        log_probs = model(user_features, product_features, prices)
        probabilities = torch.exp(log_probs)
        price_with_outside = torch.cat((torch.zeros(1, device=prices.device), prices), dim=0)
        
        total_expected_revenue = (probabilities * price_with_outside.unsqueeze(0)).sum()

    return total_expected_revenue.item()


X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)


user_features_test, product_features_test, prices_control, _ = prepare_data(X_user_test, X_product, price)

expected_revenue_control = calculate_expected_revenue(
    model, user_features_test, product_features_test, prices_control
)
print(f"Expected Revenue (Control): ${expected_revenue_control:.2f}")

all_treated_price = price * discount
user_features_test, product_features_test, prices_treated, _ = prepare_data(X_user_test, X_product, all_treated_price)

expected_revenue_treated = calculate_expected_revenue(
    model, user_features_test, product_features_test, prices_treated
)
print(f"Expected Revenue (Treated): ${expected_revenue_treated:.2f}")


Epoch 0, Training Loss: 2.5305, Validation Loss: 2.5223
Epoch 100, Training Loss: 2.2731, Validation Loss: 2.2675
Epoch 200, Training Loss: 2.2599, Validation Loss: 2.2542
Epoch 300, Training Loss: 2.2544, Validation Loss: 2.2491
Epoch 400, Training Loss: 2.2517, Validation Loss: 2.2466
Epoch 500, Training Loss: 2.2502, Validation Loss: 2.2451
Epoch 600, Training Loss: 2.2492, Validation Loss: 2.2442
Epoch 700, Training Loss: 2.2486, Validation Loss: 2.2436
Epoch 800, Training Loss: 2.2481, Validation Loss: 2.2431
Epoch 900, Training Loss: 2.2477, Validation Loss: 2.2428
Final Lambda (Nesting Parameter): 0.9620
Expected Revenue (Control): $3706.08
Expected Revenue (Treated): $763.23


In [51]:

print(f"Revenue Difference: ${2*(expected_revenue_treated - expected_revenue_control):.2f}")
nmnl = 2*(expected_revenue_treated - expected_revenue_control)
print(f"Absolute Percentage Estimation Error of PDL:  {100*np.abs(nmnl-revenue_difference)/revenue_difference:.2f}%")

Revenue Difference: $-5885.71
Absolute Percentage Estimation Error of PDL:  -0.32%


# use PDL

In [52]:
def prepare_data(user_features, product_features, prices):
    num_products = product_features.shape[0]
    all_x_other_products = []
    for i in range(num_products):
        indices = [j for j in range(num_products) if j != i]
        other_products = product_features[indices].reshape(-1)
        all_x_other_products.append(other_products)

    # Convert lists to tensor
    all_x_other_products = torch.stack(all_x_other_products, dim=0)
  

    return user_features, product_features, prices, all_x_other_products


In [53]:
X_user_train1, X_user_val, decision_train1,decision_val = train_test_split(X_user_train,decision_train,test_size=0.1,random_state=34)

In [54]:
price = price.to(device)
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount) * prod_randomization))
user_features, product_features, prices, all_x_other_products = prepared_data
user_features.shape, product_features.shape, prices.shape, all_x_other_products.shape

(torch.Size([4500, 5]),
 torch.Size([10, 5]),
 torch.Size([10]),
 torch.Size([10, 45]))

In [55]:
class PDLModel(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(PDLModel, self).__init__()
        # Combined feature dimension includes product features, price, and user features, as well as other products' features and prices
        total_feature_dim = user_feature_dim + 2*product_feature_dim + 1  # +1 for price

        # Single neural network to process the combined features
        self.network = nn.Sequential(
            nn.Linear(total_feature_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5,5),
            nn.ReLU(),
            nn.Linear(5, 1) 
        )
            # Layers to process other products' features (z-j)
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim*(NUM_Product-1), NUM_Product),
            nn.ReLU(),
            nn.Linear(NUM_Product, product_feature_dim)
        )

    def forward(self, x_user, x_product, x_other_products,prices):
        N = x_user.shape[0]
        M = x_product.shape[0]
        # Process other products' features
        aggregated_other_features = self.other_product_features_layers(x_other_products)

        
        combined_features =  torch.cat((x_user.unsqueeze(1).expand(-1, M, -1),
                                        x_product.unsqueeze(0).expand(N, -1, -1),
                                        aggregated_other_features.unsqueeze(0).expand(N, -1, -1),
                                        prices.view(1, -1, 1).expand(N, -1, -1)),
                                        dim=2)
   

        # Compute utility for each combined feature set
        utilities = self.network(combined_features).squeeze(-1)

        # Incorporate the outside option with utility 0
        zero_utilities = torch.zeros(N, 1, device=utilities.device)
        utilities_with_outside = torch.cat((zero_utilities, utilities), dim=1)

        return utilities_with_outside
        
   

In [56]:
pdlmodel = PDLModel(user_feature_dim=USER_Cont_FEATURES+USER_Dicr_FEATURES,
                       product_feature_dim=Product_Cont_FEATURES+Product_Dicr_FEATURES).to(device)

In [57]:

optimizer = torch.optim.Adam(pdlmodel.parameters(), lr=0.01)

best_val_loss = float('inf')
patience = 15
patience_counter = 0

for epoch in range(1000):
    pdlmodel.train()  
    optimizer.zero_grad()

    outputs = pdlmodel(user_features, product_features, all_x_other_products,prices)
    choice_probabilities = F.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]),decision_train1+1])


    loss.backward()
    optimizer.step()

    # Validation phase
    pdlmodel.eval()  # Set model to evaluation mode


    with torch.no_grad():
        val_outputs = pdlmodel(X_user_val,  product_features, all_x_other_products,prices)
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]),decision_val+1])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        # torch.save(pdlmodel.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break


Epoch 1, Training Loss: 2.4031901359558105, Validation Loss: 2.402212381362915
Epoch 2, Training Loss: 2.402205228805542, Validation Loss: 2.4012296199798584
Epoch 3, Training Loss: 2.4012317657470703, Validation Loss: 2.400289535522461
Epoch 4, Training Loss: 2.400249481201172, Validation Loss: 2.399254083633423
Epoch 5, Training Loss: 2.3992366790771484, Validation Loss: 2.3982481956481934
Epoch 6, Training Loss: 2.3982439041137695, Validation Loss: 2.3972465991973877
Epoch 7, Training Loss: 2.397242546081543, Validation Loss: 2.3961853981018066
Epoch 8, Training Loss: 2.396186113357544, Validation Loss: 2.3950695991516113
Epoch 9, Training Loss: 2.3950984477996826, Validation Loss: 2.3939170837402344
Epoch 10, Training Loss: 2.3939545154571533, Validation Loss: 2.392739772796631
Epoch 11, Training Loss: 2.392758369445801, Validation Loss: 2.3914601802825928
Epoch 12, Training Loss: 2.3914871215820312, Validation Loss: 2.39019513130188
Epoch 13, Training Loss: 2.3901662826538086, Val

In [58]:
import torch
import torch.nn.functional as F

def calculate_expected_revenue(model,user_features, product_features, all_x_other_products,prices):
    # Ensure model is in evaluation mode
    model.eval()

    with torch.no_grad():  # Disable gradient calculation
        utilities = model(user_features, product_features, all_x_other_products,prices)
        probabilities = F.softmax(utilities, dim=1)  # Softmax over products only

        # Calculate expected revenue for each product
        price_with_outside = torch.cat((torch.zeros(1, device=prices.device),prices), dim=0)
        total_expected_revenue = (probabilities.sum(dim=0)* price_with_outside.unsqueeze(0)).sum()


    return total_expected_revenue.item()  # Convert to Python float

In [59]:
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)
control_prepared_data = prepare_data(X_user_test, X_product,  price)
user_features, product_features, prices, all_x_other_products = control_prepared_data
# Calculate expected revenue
expected_revenue_all_control = calculate_expected_revenue(pdlmodel, user_features, product_features, all_x_other_products, prices)
print(f"Expected Revenue all Control: ${expected_revenue_all_control:.2f}")
all_treated_price = price*discount
treated_prepared_data = prepare_data(X_user_test, X_product,  all_treated_price)
user_features, product_features, prices, all_x_other_products = treated_prepared_data
expected_revenue_all_treated = calculate_expected_revenue(pdlmodel, user_features, product_features, all_x_other_products, prices)
print(f"Expected Revenue all treated: ${expected_revenue_all_treated:.2f}")

Expected Revenue all Control: $3665.69
Expected Revenue all treated: $739.75


In [60]:
pdl = (expected_revenue_all_treated-expected_revenue_all_control)*2

In [61]:
print(f"Absolute Percentage Estimation Error of PDL:  {100*np.abs(pdl-revenue_difference)/revenue_difference:.2f}%")

Absolute Percentage Estimation Error of PDL:  -0.89%


# use dml

In [62]:
class UtilityEstimator(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(UtilityEstimator, self).__init__()
        
        # Layers to process other products' features (z-j)
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim*(NUM_Product-1), NUM_Product),
            nn.ReLU(),
            nn.Linear(NUM_Product, product_feature_dim)
        )

        self.theta0 = nn.Sequential(
            nn.Linear(user_feature_dim + 2 * product_feature_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )
        # Output layer for Theta1 (takes xi, zj, z-j, p-j)
        self.theta1 = nn.Sequential(
            nn.Linear(user_feature_dim + 2 * product_feature_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )
        
    def forward(self, x_user, x_product, x_other_products,price):
        N = x_user.shape[0]
        M = x_product.shape[0]
        # Process other products' features
        aggregated_other_features = self.other_product_features_layers(x_other_products)
    

        # Combine features for Theta0
        
        combined_features_theta =  torch.cat((x_user.unsqueeze(1).expand(-1, M, -1),
                                               x_product.unsqueeze(0).expand(N, -1, -1),
                                               aggregated_other_features.unsqueeze(0).expand(N, -1, -1)),
                                                 dim=2)
        theta0_output = self.theta0(combined_features_theta).squeeze(-1)
        theta1_output = self.theta1(combined_features_theta).squeeze(-1)
        
        price = price.unsqueeze(-1)  
        utility = theta0_output + theta1_output * price.squeeze(-1)

        # Include the outside option (utility = 0)
        zero_utilities = torch.zeros(x_user.shape[0], 1, device=utility.device)
        utilities_with_outside = torch.cat((zero_utilities, utility), dim=1)
        
        return utilities_with_outside,theta0_output,theta1_output


In [63]:
dml_model = UtilityEstimator(user_feature_dim=USER_Cont_FEATURES+USER_Dicr_FEATURES,
                       product_feature_dim=Product_Cont_FEATURES+Product_Dicr_FEATURES).to(device)

In [64]:
X_user_train1, X_user_val, decision_train1,decision_val = train_test_split(X_user_train,decision_train,test_size=0.1,random_state=34)

In [65]:
def prepare_data(user_features, product_features, prices):
    num_products = product_features.shape[0]
    all_x_other_products = []
    for i in range(num_products):
        indices = [j for j in range(num_products) if j != i]
        other_products = product_features[indices].reshape(-1)
        all_x_other_products.append(other_products)

    # Convert lists to tensor
    all_x_other_products = torch.stack(all_x_other_products, dim=0)
  

    return user_features, product_features, prices, all_x_other_products


In [66]:
price = price.to(device)
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount) * prod_randomization))
user_features, product_features, prices, all_x_other_products = prepared_data
user_features.shape, product_features.shape, prices.shape, all_x_other_products.shape

(torch.Size([4500, 5]),
 torch.Size([10, 5]),
 torch.Size([10]),
 torch.Size([10, 45]))

In [67]:
import torch.nn.functional as F
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.01)

best_val_loss = float('inf')
patience = 15
patience_counter = 0

for epoch in range(1000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()
    
    outputs = dml_model(user_features, product_features, all_x_other_products,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), decision_train1+1 ])

    loss.backward()
    optimizer.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val,  product_features, all_x_other_products,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]),decision_val+1])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break


Epoch 1, Training Loss: 2.4104995727539062, Validation Loss: 2.4049038887023926
Epoch 2, Training Loss: 2.404656410217285, Validation Loss: 2.400033712387085
Epoch 3, Training Loss: 2.39985728263855, Validation Loss: 2.3957762718200684
Epoch 4, Training Loss: 2.395529270172119, Validation Loss: 2.391464948654175
Epoch 5, Training Loss: 2.3912882804870605, Validation Loss: 2.386776924133301
Epoch 6, Training Loss: 2.3867173194885254, Validation Loss: 2.381265640258789
Epoch 7, Training Loss: 2.3815460205078125, Validation Loss: 2.374903917312622
Epoch 8, Training Loss: 2.3757407665252686, Validation Loss: 2.36799693107605
Epoch 9, Training Loss: 2.3691470623016357, Validation Loss: 2.360262870788574
Epoch 10, Training Loss: 2.3618290424346924, Validation Loss: 2.3522448539733887
Epoch 11, Training Loss: 2.353926181793213, Validation Loss: 2.343076705932617
Epoch 12, Training Loss: 2.3449535369873047, Validation Loss: 2.3323538303375244
Epoch 13, Training Loss: 2.334913730621338, Validat

In [68]:
import torch
import torch.nn.functional as F

def calculate_expected_revenue(model,user_features, product_features, all_x_other_products,prices):
    # Ensure model is in evaluation mode
    model.eval()

    with torch.no_grad():  # Disable gradient calculation
        utilities = model(user_features, product_features, all_x_other_products,prices)[0]
        probabilities = F.softmax(utilities, dim=1)  # Softmax over products only

        # Calculate expected revenue for each product
        price_with_outside = torch.cat((torch.zeros(1, device=prices.device),prices), dim=0)
        total_expected_revenue = (probabilities.sum(dim=0)* price_with_outside.unsqueeze(0)).sum()


    return total_expected_revenue.item()  # Convert to Python float

In [69]:
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)
control_prepared_data = prepare_data(X_user_test, X_product,  price)
user_features, product_features, prices, all_x_other_products = control_prepared_data
# Calculate expected revenue
expected_revenue_all_control = calculate_expected_revenue(dml_model, user_features, product_features, all_x_other_products, prices)
print(f"Expected Revenue all Control: ${expected_revenue_all_control:.2f}")
all_treated_price = price*discount
treated_prepared_data = prepare_data(X_user_test, X_product,  all_treated_price)
user_features, product_features, prices, all_x_other_products = treated_prepared_data
expected_revenue_all_treated = calculate_expected_revenue(dml_model, user_features, product_features, all_x_other_products, prices)
print(f"Expected Revenue all treated: ${expected_revenue_all_treated:.2f}")

Expected Revenue all Control: $3715.34
Expected Revenue all treated: $758.80


In [70]:
expected_revenue_all_treated-expected_revenue_all_control

-2956.5409545898438

# debias the GTE estimator:

In [71]:
test_prepared_data = prepare_data(X_user_test, X_product,  price*(discount*prod_randomization))
user_features, product_features, prices, all_x_other_products = test_prepared_data

# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products,prices)


In [72]:
theta1_output.shape

torch.Size([5000, 10])

# use formulation debias for H_i

In [73]:
def H_theta(theta0_output,theta1_output,all_treated_price,price):
    N = theta0_output.shape[0]
    M = NUM_Product
    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = all_treated_price.unsqueeze(0).expand(N, M)
    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti =  theta0_output + theta1_output * expand_price


    # Include the outside option (utility = 0)
    zero_utilities = torch.zeros(N, 1, device=all_treated_uti.device)
    all_treated_uti = torch.cat((zero_utilities,all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities,all_control_uti), dim=1)

    all_treated_probabilities = F.softmax(all_treated_uti, dim=1)
    all_control_probabilities = F.softmax(all_control_uti, dim=1)

    price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
    treated_price_with_outside =  torch.cat((torch.zeros(1, device=all_treated_price.device),all_treated_price), dim=0)

    H = torch.sum(all_treated_probabilities*treated_price_with_outside - all_control_probabilities*price_with_outside,dim=1)
    expsum_treated = torch.sum(torch.exp(all_treated_uti),dim=1)
    expsum_control = torch.sum(torch.exp(all_control_uti),dim=1)

    expsum_treated_expanded = expsum_treated.unsqueeze(1).expand(-1, all_treated_uti.shape[1])  # Shape [N, M+1]
    expsum_control_expanded = expsum_control.unsqueeze(1).expand(-1, all_control_uti.shape[1])  # Shape [N, M+1]

    H_theta0 = torch.sum((torch.exp(all_treated_uti)*(1-torch.exp(all_treated_uti))/expsum_treated_expanded/expsum_treated_expanded-\
                          torch.exp(all_control_uti)*(1-torch.exp(all_control_uti))/expsum_control_expanded/expsum_control_expanded)\
                         *price_with_outside,dim=1)
    H_theta1 = torch.sum(price_with_outside*(torch.exp(all_treated_uti)*(1-torch.exp(all_treated_uti))/expsum_treated_expanded/expsum_treated_expanded*treated_price_with_outside-\
                                             torch.exp(all_control_uti)*(1-torch.exp(all_control_uti))/expsum_control_expanded/expsum_control_expanded*price_with_outside),dim=1)


    return H,H_theta0,H_theta1


In [74]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

In [75]:
def l_theta(theta0_output,theta1_output,adjusted_price,decision_test):
    N = theta0_output.shape[0]
    M = NUM_Product
    expand_adjusted_price = adjusted_price.unsqueeze(0).expand(N, M)
    uti = theta0_output + theta1_output * expand_adjusted_price
    adjusted_price_with_outside =  torch.cat([torch.zeros(1, device=adjusted_price.device),adjusted_price])

    # Include the outside option (utility = 0)
    zero_utilities = torch.zeros(N, 1, device=uti.device)
    uti = torch.cat((zero_utilities,uti), dim=1)

    probabilities = F.softmax(uti, dim=1)
    prod_indices = torch.ones(NUM_Product, device=device)
    prod_indices = torch.cat([torch.zeros(1,device=device),prod_indices])
    ltheta0 = probabilities[torch.arange(decision_test.size(0)), decision_test+1] -prod_indices[decision_test+1]
    ltheta1 = (probabilities[torch.arange(decision_test.size(0)), decision_test+1] * adjusted_price_with_outside[decision_test+1]) - adjusted_price_with_outside[decision_test+1]


    return ltheta0,ltheta1

In [76]:
price = price.to(device)

In [77]:
adjusted_price = price*(discount*prod_randomization).to(device)
decision_test = decision_test.to(device)
ltheta0,ltheta1= l_theta(theta0_output,theta1_output,adjusted_price,decision_test)

In [78]:
import torch
import torch.nn.functional as F

def lambdainv(theta0_output, theta1_output, price, decision_test,epsilon =10):
    N = theta0_output.shape[0]
    M = NUM_Product
    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = discount*price.unsqueeze(0).expand(N, M)

    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti =  theta0_output + theta1_output * expand_price

    # Include the outside option (utility = 0)
    zero_utilities = torch.zeros(N, 1, device=all_control_uti.device)
    all_treated_uti = torch.cat((zero_utilities,all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities,all_control_uti), dim=1)

    # Calculate probabilities using softmax
    probabilities_control = F.softmax(all_control_uti, dim=1)
    probabilities_treated = F.softmax(all_treated_uti, dim=1)

    # Extract probabilities of chosen products
    chosen_prob_control = probabilities_control[torch.arange(N), decision_test]
    chosen_prob_treated = probabilities_treated[torch.arange(N), decision_test]

    # Calculate second derivatives
    ltheta00 = -chosen_prob_control * (1 - chosen_prob_control) - chosen_prob_treated * (1 - chosen_prob_treated)
    ltheta01 = -chosen_prob_control * (1 - chosen_prob_control) * expand_price[torch.arange(N), decision_test] - \
            chosen_prob_treated * (1 - chosen_prob_treated) * (discount * expand_price[torch.arange(N), decision_test])
    ltheta11 = -chosen_prob_control * (1 - chosen_prob_control) * expand_price[torch.arange(N), decision_test]**2 - \
            chosen_prob_treated * (1 - chosen_prob_treated) * (discount * expand_price[torch.arange(N), decision_test])**2
    ltheta00=ltheta00/2
    ltheta01=ltheta01/2
    ltheta11=ltheta11/2

    # Form the 2x2 Hessian matrices for each instance
    ltheta00 = ltheta00.unsqueeze(1).unsqueeze(2)
    ltheta01 = ltheta01.unsqueeze(1).unsqueeze(2)
    ltheta11 = ltheta11.unsqueeze(1).unsqueeze(2)

    top_row = torch.cat((ltheta00, ltheta01), dim=2)
    bottom_row = torch.cat((ltheta01, ltheta11), dim=2)

    L_matrix = torch.cat((top_row, bottom_row), dim=1)

    # Regularization and inversion
    
    identity_matrix = torch.eye(2, dtype=L_matrix.dtype, device=L_matrix.device) * epsilon
    L_matrix_reg = L_matrix + identity_matrix.unsqueeze(0).unsqueeze(0)
    L_inv = torch.linalg.inv(L_matrix_reg)

    return L_inv


In [79]:
epsilon_list = [0.001,0.01,0.1,0.5,1,5,10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, decision_test, epsilon).float()
    
        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()  
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()  
    
        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0)) 
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)  
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0
    
        # Calculate sdl and dedl
        sdl = H.sum().cpu().detach().numpy() * 2
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 2
    
        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)
    
        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

In [80]:
sdl = H.sum().cpu().detach().numpy()*2

In [81]:
dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy())*2

In [82]:
sdl,dedl,best_epsilon

(-5913.08203125, -5905.2841796875, 5)

In [83]:
print(f"Absolute Percentage Estimation Error of SDL:  {100*np.abs(sdl-revenue_difference)/revenue_difference:.2f}%")
print(f"Absolute Percentage Estimation Error of SP MNL:  {100*np.abs(dedl-revenue_difference)/revenue_difference:.2f}%")

Absolute Percentage Estimation Error of SDL:  -0.15%
Absolute Percentage Estimation Error of SP MNL:  -0.01%


In [84]:
naive_pe = (naive - true) / true
linear_pe = (linear - true) / true
pdl_pe = (pdl - true) / true
sdl_pe = (sdl - true) / true
dedl_pe = (dedl - true) / true
naive_mse = (naive - true)**2
linear_mse =(linear - true)**2
pdl_mse = (pdl - true)**2
sdl_mse = (sdl - true)**2
dedl_mse = (dedl - true)**2
naive_e = (naive - true)
linear_e =(linear - true)
pdl_e = (pdl - true)
sdl_e = (sdl - true)
dedl_e = (dedl - true)

In [85]:
print(naive_pe,linear_pe,pdl_pe,sdl_pe,dedl_pe,naive_e,linear_e,pdl_e,sdl_e,dedl_e,naive_mse,linear_mse,pdl_mse,sdl_mse,dedl_mse)

-0.6562964572817541 -0.0013449696870614395 -0.008908661759576532 0.0014568786134413194 0.00013621165086556724 3875.089254066348 7.941346511244774 52.601014479994774 -8.602110520005226 -0.8042589575052261 15016316.726980485 63.064984411659545 2766.86672432462 73.99630539838458 0.6468324707273931
